# Migrating from Cards to Tabs macro

This script migrates from the currently used used [Deck of Cards / Card macro](https://apps-docs.servicerocket.com/composition/deck-of-cards) (which is deprecated) to the [Tabs Container / Tabs macro](https://docs.adaptavist.com/cfm4cs/latest/content-formatting-macros/tabs)

The Tabs macro brings two primary benefits:
* Lazy loading means content is not generated unless somebody is actually looking at the respective tab. This leads to major performance improvements for Pages using the [Page Properties Report](https://confluence.atlassian.com/doc/page-properties-report-macro-186089616.html) macro (which in turn relies on the Page Properties macro on the affected pages).
* The Tabs Macro supports directly linking to a specific tab out of the box, which makes it easier to share a specific section of a given page.

List all supoprted magic commands

In [ ]:
#%lsmagic

Install Atlassian API

In [ ]:
#%pip install atlassian-python-api
#%pip install lxml

### Enterprise Technology Solutions / Services Confluence Space Admins
Ben Murphy, Brian Cunningham, Calvin Mccoy Jr., Confluence Role account, Edwin Wee, Jianming Ling, Lance Wisdom, Macy Liu, Mandy Jia Man Lin, Maninder Singh, Mikio Ichino, Sam Lee, Simon Michael, Stuart Cresp, Thomas Warner, Zhendong Fu 

## Import modules

In [ ]:
from atlassian import Confluence #https://atlassian-python-api.readthedocs.io/
from bs4 import BeautifulSoup #https://www.crummy.com/software/BeautifulSoup/bs4/doc/
import os
import requests
import traceback
import tqdm.notebook as tq

### Step 1 - Change the Card Deck to a Tabs Container

In [ ]:
def change_Deck_to_TabsContainer(content) -> str:
    return content.replace('ac:name="deck"', 'ac:name="auitabs"')

### Step 2 - Change Cards to Tabs Pages

In [ ]:
def find_all_substrings(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

In [ ]:
def change_Cards_to_TabsPage(content) -> str:
    content = content.replace('ac:name="card"', 'ac:name="auitabspage"')
    indices = list(find_all_substrings(content, 'ac:name="auitabspage"')) #60
    
    lookahead_length = 1000 #How many characters ahead we look for 'label' after 'auitabspage' to avoid conflict with other elements using 'label'
    
    for idx in indices:
        temp = content[idx: (idx + lookahead_length)]
        temp = temp.replace('ac:name="label"', 'ac:name="title"')
        #print(temp + '\n')
        content = "".join((content[:idx], temp, content[idx + lookahead_length:]))
    
    return content

### Find macros that are incompatible with lazy loading

In [ ]:
def incompatible_plugin(element) -> bool:
    if any([
        element.find(attrs={"ac:name": "details"}),             #Page Properties, because of Page Properties Report
        element.find(attrs={"ac:name": "excerpt-include"}),     #Excerpt Include
        element.find(attrs={"ac:name": "multiexcerpt-include"}),#Multi-Excerpt Include
        element.find(attrs={"ac:name": "include"}),             #Page Include
        element.find(attrs={"ac:name": "expand"}),              #Expand (regular)
        element.find(attrs={"ac:name": "ui-expand"}),           #UI Expand
        element.find(attrs={"ac:name": "viewxls"}),             #Excel Viewer
        element.find(attrs={"ac:name": "viewdoc"}),             #Word Viewer
        element.find(attrs={"ac:name": "viewppt"}),             #PowerPoint Viewer
        element.find(attrs={"ac:name": "viewpdf"}),             #PowerPoint Viewer
        element.find("ac:task"),                                #Task List (Checkboxes)
        ]):
        return True
    return False

### Step 3 - Apply lazy loading where possible

In [ ]:
def add_lazy_loading(element, soup, depth) -> None:
    #print(depth, '\n', element, '\n')
    depth += 1
    update = False
    sub_element = element.find(attrs={"ac:name": "auitabspage"})
    
    if not sub_element: #Base case
        if not incompatible_plugin(element):
            new_tag = soup.new_tag("ac:parameter")
            new_tag.string = "true"
            new_tag["ac:name"] = "lazyloading"
            before = element.find(attrs={"ac:name": "title"})
            before.insert_after(new_tag)
    else:
        add_lazy_loading(sub_element, soup, depth)

def process_lazy_loading(content) -> None:
    all_structured_macro_elements = content.find_all(attrs={"ac:name": "auitabspage"})
    for macro in all_structured_macro_elements:
        add_lazy_loading(macro, content, 1)

### Process each page in sequence

In [ ]:
def process_page(content) -> str:
    content = change_Deck_to_TabsContainer(content)
    content = change_Cards_to_TabsPage(content)
    soup = BeautifulSoup(content, 'lxml')
    process_lazy_loading(soup)
    content = "".join([str(x) for x in soup.body.children])
    return content

### Set up the Confluence API

In [ ]:
username = 'etec_team_api'
#passwd = getpass.getpass('Password: ')
passwd = os.environ['etec_team_api']

confluence = Confluence(
    #url='https://cms.alpha.bloomberg.com/team', #ALPHA
    url='https://cms.prod.bloomberg.com/team', #PROD
    username=username,
    password=passwd,
    proxies={'http': 'http://bproxy.tdmz1.bloomberg.com:80', 'https': 'http://bproxy.tdmz1.bloomberg.com:80'},
    #verify_ssl='F://bb-cert//bloomberg-root-ca.crt',
)

### Store problematic pages
Rather than erroring out of we encounter an exception, we store problematic pages and the exception here to be looked at manually later

In [ ]:
pages_with_errors = {} #Dictionary 

### Get pages in scope
Those pages are children of the AMER, APAC and EMEA regional pages, as well as the generic pages. I'm working through manually.

In [ ]:
# parent = 658768638 #https://cms.prod.bloomberg.com/team/display/tsci/AMER+Region
# parent = 658768579 #https://cms.prod.bloomberg.com/team/display/tsci/APAC+Region
# parent = 658768576 #https://cms.prod.bloomberg.com/team/display/tsci/EMEA+Region
# parent = 2067235343 #https://cms.prod.bloomberg.com/team/display/tsci/Generic+%28Multi-tenant%29+Gateways

children = confluence.get_page_child_by_type(parent, type='page', start=None, limit=None, expand=None)
pages = [int(x['id']) for x in children]

### Start the actual page updates

In [ ]:
# Problematic pages
# pages = [3124080081] #Error with embedded XML specs

for page_id in tq.tqdm(pages):
    try: 
        # print("Load page " + str(page_id))
        page = confluence.get_page_by_id(page_id, expand='body.storage')
        title = page['title']
        content = page['body']['storage']['value']
        # print("Process page " + str(page_id))
        content = process_page(content)
        print("Update page " + str(page_id))
        confluence.update_page(page_id, title=title, body=content )
    except Exception as e:
        pages_with_errors[page_id] = str(traceback.format_exc())

### Print any pages with errors

In [ ]:
error_page_ids = list(pages_with_errors.keys())
print(error_page_ids)
print(pages_with_errors)

### For testing

In [ ]:
# Test xml

source = '''
<ac:structured-macro ac:name="deck" >
  <ac:parameter ac:name="id">Handover-Template</ac:parameter>
  <ac:rich-text-body>
    <ac:structured-macro ac:name="card">
      <ac:parameter ac:name="label">Project History</ac:parameter>
      <ac:rich-text-body>
        <p>Content</p>
      </ac:rich-text-body>
    </ac:structured-macro>
    <ac:structured-macro ac:name="card">
      <ac:parameter ac:name="label">Tab2</ac:parameter>
      <ac:rich-text-body>
        <ac:structured-macro ac:name="deck">
          <ac:parameter ac:name="id">Deck2</ac:parameter>
          <ac:rich-text-body>
            <ac:structured-macro ac:name="card">
              <ac:parameter ac:name="label">Tab3</ac:parameter>
              <ac:rich-text-body>
                <p>Content</p>
              </ac:rich-text-body>
            </ac:structured-macro>
          </ac:rich-text-body>
        </ac:structured-macro>
      </ac:rich-text-body>
    </ac:structured-macro>
  </ac:rich-text-body>
</ac:structured-macro>
'''

target = '''
<ac:structured-macro ac:name="auitabs">
  <ac:parameter ac:name="id">Project History</ac:parameter>
  <ac:rich-text-body>
    <ac:structured-macro ac:name="auitabspage">
      <ac:parameter ac:name="lazyloading">true</ac:parameter>
      <ac:parameter ac:name="id">Tab2</ac:parameter>
      <ac:parameter ac:name="title">Tab2</ac:parameter>
      <ac:rich-text-body>
        <p>Content</p>
      </ac:rich-text-body>
    </ac:structured-macro>
    <ac:structured-macro ac:name="auitabspage">
      <ac:parameter ac:name="id">Tab3</ac:parameter>
      <ac:parameter ac:name="title">Tab3</ac:parameter>
      <ac:rich-text-body>
        <ac:structured-macro ac:name="auitabs">
          <ac:parameter ac:name="id">tc2</ac:parameter>
          <ac:rich-text-body>
            <ac:structured-macro ac:name="auitabspage" >
              <ac:parameter ac:name="lazyloading">true</ac:parameter>
              <ac:parameter ac:name="id">Tab3</ac:parameter>
              <ac:parameter ac:name="title">Tab3</ac:parameter>
              <ac:rich-text-body>
                <p>Content</p>
              </ac:rich-text-body>
            </ac:structured-macro>
          </ac:rich-text-body>
        </ac:structured-macro>
      </ac:rich-text-body>
    </ac:structured-macro>
  </ac:rich-text-body>
</ac:structured-macro>
'''
content = source

# content = source
# content = change_Deck_to_TabsContainer(content)
# content = change_Cards_to_TabsPage(content)
# print(content)
# soup = BeautifulSoup(content, 'lxml')
# process_lazy_loading(soup)
# content = "".join([str(x) for x in soup.body.children])
# print(content)